In [1]:
import sys
sys.path.append('../')

In [2]:
import os, glob, random, cv2
import wandb
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import albumentations as A
import segmentation_models_pytorch as smp
import model.metric as module_metric

from data_loader.dataloader import get_dataloader 
from utils.data import get_datasize
from utils.visual import *
from albumentations.pytorch import transforms
from model.loss import *
from train import *
from pathlib import Path

In [3]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

lr = 1e-3
batch_size = 4
num_epoch = 100

train_dir = './dataset/unrefined/crushed/train/'
val_dir = './dataset/unrefined/crushed/val/'

In [4]:
transform_train = A.Compose([
    A.HorizontalFlip(),
    A.Rotate((-30, 30), p=0.5, border_mode=cv2.BORDER_REFLECT,),
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3),
    A.Resize(256, 256),
    A.Normalize(mean = 0.5, std=0.5),
    transforms.ToTensorV2(transpose_mask=True)
])

transform_val = A.Compose([
    A.Resize(256, 256),
    A.Normalize(mean = 0.5, std=0.5),
    transforms.ToTensorV2(transpose_mask=True)
])

In [5]:
train_dataloader = get_dataloader(train_dir, transform_train, batch_size)
val_dataloader = get_dataloader(val_dir, transform_val, batch_size)

In [6]:
model = smp.Unet(encoder_name='efficientnet-b7', encoder_weights='imagenet', in_channels=3, classes=1, activation=None)
model = model.to(device)

In [7]:
criterion = DiceLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='min', factor=0.1, patience=10, min_lr=1e-6)

In [8]:
train_config = {}

In [9]:
train_config['Batch size'] = batch_size
train_config['Learning Rate'] = lr
train_config['Epochs'] = num_epoch

train_config['Loss fn'] = 'Dice'
train_config['Optimizer'] = 'SGD'
train_config['LR Scheduler'] = 'ReduceLROnPlateau'
train_config['Metric'] = [metric for metric in ['IOUscore', 'PixelAccuracy']]

In [10]:
wandb.init(project='Segmentation', name='UNet', config=train_config)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: imlim. Use `wandb login --relogin` to force relogin


In [11]:
metrics = [getattr(module_metric, met) for met in ['IOUscore', 'PixelAccuracy']]

In [12]:
trainer = Trainer(model, criterion, metrics, optimizer, device, num_epoch, Path('./saved/Efficient_B0'),
                                  data_loader=train_dataloader, valid_data_loader=val_dataloader,
                                  lr_scheduler=scheduler)

In [13]:
trainer.train()


Epoch : 0 | 

KeyboardInterrupt: 